# Frequent Pattern Mining

## Instructions
### 1 Market Basket Analysis
Compute:
• Frequency of each itemset.
• Support of each itemset.
• Confidence of association rules.
• Lift of association rules.



# 1
First I need, I (list of items), Then D (dataset)

Then I need to create a sublist of 2 items

Then I need to see how many times said list exactly shows up in dataset

Then I need to see how many times said list items show up in dataset



In [1]:
import pandas as pd
import numpy as np
import math
df = pd.read_csv('basket.csv')

Elementos=[]
df["Items"] = df["Items"].str.replace(" ", "", regex=True)
df["Items"].apply(lambda x:[Elementos.append(i) for i in x.split(",")])
print(Elementos)
#display(df)

['bread', 'eggs', 'milk', 'cheese', 'beer', 'cheese', 'milk', 'diapers', 'milk', 'cheese', 'beer', 'bread', 'diapers', 'eggs', 'butter', 'bread', 'bread', 'diapers', 'milk', 'bread', 'diapers', 'bread', 'cheese', 'beer', 'eggs', 'diapers', 'diapers', 'butter', 'cheese', 'beer', 'butter', 'eggs', 'cheese', 'milk', 'bread', 'butter', 'milk', 'diapers', 'milk', 'butter', 'diapers', 'eggs', 'diapers', 'butter', 'bread', 'milk', 'beer', 'butter', 'milk', 'bread', 'butter', 'eggs', 'milk', 'diapers', 'butter', 'bread', 'bread', 'beer', 'diapers', 'milk', 'eggs', 'beer', 'milk', 'butter', 'diapers', 'eggs', 'bread', 'beer', 'beer', 'milk', 'bread', 'butter', 'beer', 'cheese', 'bread', 'eggs', 'eggs', 'beer', 'milk', 'cheese', 'eggs', 'butter', 'cheese', 'bread', 'eggs', 'eggs', 'bread', 'beer', 'milk', 'eggs', 'milk', 'diapers', 'bread', 'cheese', 'butter', 'bread', 'eggs', 'diapers', 'cheese', 'butter', 'bread', 'eggs', 'diapers', 'beer', 'bread', 'milk']


In [2]:
#several support functions, first for two items intersec, for "Only association rules between one-item sets (e.g., item1 → item2) should be considered for this task"

def support_calc(df, column ,element_a, element_b):
    counter=df[column].apply(lambda x: element_a in x  and element_b in x)
    counter=counter.sum()
    #print(counter)
    return counter

#one for the union because i will need it for association rules
def support_union_calc(df, column ,element_a, element_b):
    counter=df[column].apply(lambda x: element_a in x  or element_b in x)
    counter=counter.sum()
    #print(counter)
    return counter

#one for the individual because i will need it for association rules
def support_single_calc(df, column ,element_a):
    counter=df[column].apply(lambda x: element_a in x)
    counter=counter.sum()
    #print(counter)
    return counter

support_calc(df,"Items" ,"eggs", "cheese") #testing

7

In [3]:
#frequency no longer needed
#----------------------------

def frequency_calc(df, column ,element_a, element_b):
    frequency=0
    element_a_b_aux = [element_a, element_b]
    #print(element_a_b_aux)
    counter={}
    for element in set(element_a_b_aux):
        counter[element] = element_a_b_aux.count(element)
    #print("to find",counter)
    for element in df[column]:
        elements_row=[]
        elements_row.extend(element.split(","))
        #print(elements_row)
        counter_row = {}
        for element in set(elements_row):
            counter_row[element] = elements_row.count(element)
        #print("find in",counter_row)
        if counter_row==counter:
            frequency=frequency+1
    #print(frequency)
    return frequency


#for more than two items
def frequency_calc_exp(df, column ,element_list):
    frequency=0
    counter={}
    for element in set(element_list):
        counter[element] = element_list.count(element)
    for element in df[column]:
        elements_row=[]
        elements_row.extend(element.split(","))
        #print(elements_row)
        counter_row = {}
        for element in set(elements_row):
            counter_row[element] = elements_row.count(element)
        #print("find in",counter_row)
        if counter_row==counter:
            frequency=frequency+1
    #print(frequency)
    return frequency
#print(frequency_calc(df,"Items" ,"beer", "milk")) #testing

#print(frequency_calc_exp(df, "Items", ["beer", "milk"]))

#-----------------------------


Now i iterate through each combination and get their frequencies and support

In [4]:
#first i will create an array of all combos of 2 items so that then i can add their union support
all_combinations=[]

for i in range(len(Elementos)):
    for j in range(i+1, len(Elementos)): 
        all_combinations.append((Elementos[i], Elementos[j]))
print(all_combinations)
print(len(all_combinations))

combinations_n =int(math.factorial(len(Elementos))/(math.factorial(2)*math.factorial(len(Elementos)-2)))
print(combinations_n)

[('bread', 'eggs'), ('bread', 'milk'), ('bread', 'cheese'), ('bread', 'beer'), ('bread', 'cheese'), ('bread', 'milk'), ('bread', 'diapers'), ('bread', 'milk'), ('bread', 'cheese'), ('bread', 'beer'), ('bread', 'bread'), ('bread', 'diapers'), ('bread', 'eggs'), ('bread', 'butter'), ('bread', 'bread'), ('bread', 'bread'), ('bread', 'diapers'), ('bread', 'milk'), ('bread', 'bread'), ('bread', 'diapers'), ('bread', 'bread'), ('bread', 'cheese'), ('bread', 'beer'), ('bread', 'eggs'), ('bread', 'diapers'), ('bread', 'diapers'), ('bread', 'butter'), ('bread', 'cheese'), ('bread', 'beer'), ('bread', 'butter'), ('bread', 'eggs'), ('bread', 'cheese'), ('bread', 'milk'), ('bread', 'bread'), ('bread', 'butter'), ('bread', 'milk'), ('bread', 'diapers'), ('bread', 'milk'), ('bread', 'butter'), ('bread', 'diapers'), ('bread', 'eggs'), ('bread', 'diapers'), ('bread', 'butter'), ('bread', 'bread'), ('bread', 'milk'), ('bread', 'beer'), ('bread', 'butter'), ('bread', 'milk'), ('bread', 'bread'), ('bread

In [5]:
#to each potential combination i will add a column of 1s, which will be then replace by their support
combinations_array = np.array(all_combinations)

#this one will have individual support
combinations_array_sup = np.hstack((combinations_array, np.ones((combinations_array.shape[0], 1))))

#this one is no longer needed
combinations_array_freq = np.hstack((combinations_array, np.ones((combinations_array.shape[0], 1))))

#this one will have union support
combinations_array_union_sup = np.hstack((combinations_array, np.ones((combinations_array.shape[0], 1))))

#as a reminder, Elementos has my individual items, Elementos_array will have also support
Elementos_array = np.array(Elementos)
Elementos_array=Elementos_array.reshape(-1,1)
Elementos_array = np.hstack((Elementos_array, np.ones((Elementos_array.shape[0],Elementos_array.shape[1]))))
#print(Elementos_array)

print("for two items intersection ",np.shape(combinations_array_sup))
print("for two items union ",np.shape(combinations_array_union_sup))
print("for one item ",np.shape(Elementos_array))

for two items intersection  (5565, 3)
for two items union  (5565, 3)
for one item  (106, 2)


In [6]:
#here i add to combinations_array_union_sup its corresponding union supports
for i in range(combinations_array_union_sup.shape[0]):
    combinations_array_union_sup[i, -1] = support_union_calc(df, 'Items', combinations_array_union_sup[i, 0], combinations_array_union_sup[i, 1])

#here i add to Elementos_array its corresponding individual supports
for i in range(Elementos_array.shape[0]):
    Elementos_array[i, -1] = support_single_calc(df, 'Items', Elementos_array[i, 0])

#testing
print(combinations_array_union_sup[12])
print(Elementos_array[12])
#print(Elementos_array)
print(np.shape(Elementos_array))
#print(combinations_array_union_sup)

['bread' 'eggs' '24']
['diapers' '15']
(106, 2)


In [7]:
#next steps (slides)
#it is sufficient:
#1. to find all frequent itemsets that exceed a given minimum support s and their supports and then
#2. to split each frequent itemset Z in any two subsets X, Y s.t. the rule (X, Y ) meets the minimum confidence requirement.
#start with rule (Z, ∅) with confidence 1,
#iteratively move one element from body to head and retain only those rules that meet the minimum confidence requirement.


def find_assoc_rules(df, single_item_sup_array, min_s, min_c):
    #first criteria find all frequent itemsets that exceed a given minimum support s and their supports
    for i in range(single_item_sup_array.shape[0]):
        item_set_a, support_a=single_item_sup_array[i]
        support_a=float(support_a)
        #print(i, ". item_set_a", item_set_a, "---support_a", support_a)
        for j in range(i+1, single_item_sup_array.shape[0]):
            item_set_b, support_b=single_item_sup_array[j]
            #here i still need to apply a duplicate filter, like bread - bread



            #----------
            support_b=float(support_b)
            #print("    item_set_b", item_set_b, "---support_b", support_b)
            support_union_a_b=float(support_union_calc(df,"Items", item_set_a, item_set_b)   )
            #print("     support_union",support_union_a_b)         
            confidence_a_b = support_union_a_b / support_a
            #print("     confidence_a_b",confidence_a_b)         
            #now i have for each "i" item, its confidences and support with each union
            #now lets see if I need to save "i" as confident or not
            if support_union_a_b >= min_s:
                    if confidence_a_b>min_c:
                         #then here i display the association rule and compute its lift
                         lift=confidence_a_b/support_b
                         print("Rule:   X: ",item_set_a,"--> Y: ",item_set_a) 
                         print(" support_a", support_a, "and support_b ", support_b, " ,support_union_a_b ",support_union_a_b," ,confidence_a_b ",confidence_a_b," ,lift ",lift ) 
                         print("\n")
            else:
                 pass
find_assoc_rules(df, Elementos_array, 0.1, 0.1)

Rule:   X:  bread --> Y:  bread
 support_a 20.0 and support_b  16.0  ,support_union_a_b  24.0  ,confidence_a_b  1.2  ,lift  0.075


Rule:   X:  bread --> Y:  bread
 support_a 20.0 and support_b  17.0  ,support_union_a_b  26.0  ,confidence_a_b  1.3  ,lift  0.07647058823529412


Rule:   X:  bread --> Y:  bread
 support_a 20.0 and support_b  11.0  ,support_union_a_b  24.0  ,confidence_a_b  1.2  ,lift  0.10909090909090909


Rule:   X:  bread --> Y:  bread
 support_a 20.0 and support_b  13.0  ,support_union_a_b  24.0  ,confidence_a_b  1.2  ,lift  0.0923076923076923


Rule:   X:  bread --> Y:  bread
 support_a 20.0 and support_b  11.0  ,support_union_a_b  24.0  ,confidence_a_b  1.2  ,lift  0.10909090909090909


Rule:   X:  bread --> Y:  bread
 support_a 20.0 and support_b  17.0  ,support_union_a_b  26.0  ,confidence_a_b  1.3  ,lift  0.07647058823529412


Rule:   X:  bread --> Y:  bread
 support_a 20.0 and support_b  15.0  ,support_union_a_b  25.0  ,confidence_a_b  1.25  ,lift  0.083333333333

In [8]:
%reset -f

#  Apriori algorithm
1. Implement the vanilla Apriori algorithm from scratch. Print the frequent itemsets for each
support level and the corresponding association rules.
2. Implement the improved Apriori algorithm from scratch. Print the frequent itemsets for each support level
and the corresponding association rules.
3. Implement the vanilla Eclat algorithm from scratch. Print the frequent itemsets for each
support level and the corresponding association rules.


Note: Minimum support level to 0.02 and the minimum confidence level to 0.3.

## General setup

In [9]:
import pandas as pd
import numpy as np
import math

#preprocessing, load the dataset
transactions = pd.read_csv('Market_Basket_Optimisation.csv', header=None)
#transactions = transactions.sample(n=100, random_state=1) #sample

#now for each row convert to list without nans and then make it all a list
transactions = transactions.apply(lambda x: x.dropna().tolist(), axis=1).tolist()
#retrospect comment: need to remove NaN or these overflow my transactions

print(len(transactions))
print(len(transactions))
print(len(transactions[0]))



7501
7501
20


In [10]:
#the only purpose of this cell was testing, that's why its decontinued, but showed same behavior as supplementary material

transactions_decontinued = [{"milk", "bread", "eggs", "butter"}, {"milk", "bread", "butter"}, {"milk", "bread"}, {"bread", "eggs"}, {"bread", "eggs", "butter"}, {"milk", "butter"}, {"bread", "butter"}]


itemsets_decontinued = [{'eggs'}, {'milk'}, {'butter'}, {'bread'}]


In [11]:
#now get items, go through each row in transactions and for each row go item by item
uniques=set(item for row in transactions for item in row)
itemsets=[{item} for item in uniques]
print(itemsets)

[{'soup'}, {'whole wheat pasta'}, {'shallot'}, {'mushroom cream sauce'}, {'salt'}, {'zucchini'}, {'nonfat milk'}, {'ham'}, {'rice'}, {'whole wheat rice'}, {'mayonnaise'}, {'soda'}, {'cottage cheese'}, {'pancakes'}, {'pet food'}, {'frozen vegetables'}, {'cookies'}, {'black tea'}, {'cider'}, {'chocolate bread'}, {'shampoo'}, {'parmesan cheese'}, {'mashed potato'}, {'french wine'}, {'carrots'}, {'chutney'}, {'almonds'}, {'gums'}, {'burger sauce'}, {'napkins'}, {'light cream'}, {'fresh tuna'}, {'honey'}, {'water spray'}, {'olive oil'}, {'antioxydant juice'}, {'whole weat flour'}, {'pickles'}, {'tomato juice'}, {'strong cheese'}, {'spaghetti'}, {'sparkling water'}, {'champagne'}, {'energy drink'}, {' asparagus'}, {'green beans'}, {'yams'}, {'ground beef'}, {'mint'}, {'herb & pepper'}, {'body spray'}, {'hot dogs'}, {'tomatoes'}, {'cake'}, {'cereals'}, {'frozen smoothie'}, {'strawberries'}, {'pasta'}, {'mint green tea'}, {'cooking oil'}, {'fresh bread'}, {'light mayo'}, {'butter'}, {'cream'},

## Apriori

In [12]:
#here i just create the functions and test them
def support_finder(df, itemset):
    #i check how many rows are the same in the df as in the itemset
    counter=0
    for row in df:
        if all(item in row for item in itemset): #if all true that every item in row is in items in dataset
            counter=counter+1
    return counter
#testing

print("support",support_finder(transactions, ["honey", "pasta" ]))

#------------------------------confidence finder new addhition

def find_confident_itemsets(df, combinations, min_conf):
    confidents = []
    for itemset in combinations:
        #print(itemset)
        if len(itemset) > 1:
            support_union_items = support_finder(df, itemset)
            pre_arrow = list(itemset)[:-1]
            support_pre_arrow = support_finder(df, pre_arrow)
            if support_pre_arrow > 0:
                confidence = support_union_items / support_pre_arrow
                if confidence >= min_conf:
                    confidents.append(itemset)
    return confidents

#print("confident",find_confident_itemsets(transactions, [{"honey"}, {"pasta"}, {"honey","pasta"} ], 0))
print("confident",find_confident_itemsets(transactions, itemsets, 0))

#------------------------------------------------------------------------



def find_frequent_itemsets(df, combinations, min_support):
    frequents=[]
    frequents_support=[]
    for itemset in combinations:
        support_sum = support_finder(df,itemset)
        support_sum, support_aux=support_sum, support_sum
        #print(itemset, ";", support_sum,";",support_sum>=min_support )
        if support_sum>=min_support:
            frequents.append(itemset)
            frequents_support.append(support_sum)
    return frequents, frequents_support

#print("Frequent")
#find_frequent_itemsets(transactions, itemsets, 150)



def create_next_level(prev_itemsets):
    next_level = []
    for i in range(len(prev_itemsets)):
        # print(i,"----")
        for j in range(i + 1, len(prev_itemsets)):  #avoid dups
            new_combo = prev_itemsets[i] | prev_itemsets[j] #note: if using itemsets with sets this works, if lists use rev_itemsets[i]+prev_itemsets[j]
            #print(new_combo)
            if len(new_combo) == len(prev_itemsets[0]) + 1: #filter out bad size
                    if not any(new_combo == itemset for itemset in next_level):
                        next_level.append(new_combo)
    #print("prev", prev_itemsets, " /// ", "next", next_level, "\n")
    return next_level


def comparator(lista, listb):
    new_list = lista.copy()  
    for items in listb:
        if items not in new_list:
            new_list.append(items)
    return new_list

#will do it alphabetically
def apriori_improvement(previous_itemsets, next_itemsets):
    itemsets = []
    next_itemsets=sorted(next_itemsets, key=lambda x: "".join(sorted(x)))
    for prev_set in previous_itemsets:
        prev_set_sorted = sorted(prev_set)
        for next_set in next_itemsets:
            if prev_set_sorted==sorted(next_set-{next_set.difference(prev_set_sorted).pop()}):
                itemsets.append(next_set)
                break
    return itemsets



#apriori_improvement([{'milk', 'bread'}, {'beer', 'honey'}], [{'milk', 'bread', 'b'}, {'milk', 'bread', 'a'}, {'milk', 'bread', 'c'},{'beer', 'honey', 'a'}, {'beer', 'honey', 'b'}, {'beer', 'honey', 'c'}])
#print(itemsets)
#create_next_level(itemsets)
#create_next_level(create_next_level(itemsets))

support 7
confident []


### Vanilla apriori

In [13]:
previous_itemsets=itemsets
forced_breaker=0
while True:
    print("Level",forced_breaker, "------------------------------------------------")
    frequent_itemsets, frequent_itemsets_values=find_frequent_itemsets(transactions, previous_itemsets, 150)
    confident_itemsets=find_confident_itemsets(transactions, frequent_itemsets, 0.3)
    if confident_itemsets==[]:
        pass
    else:
        frequent_itemsets=confident_itemsets
        
    for itemset in frequent_itemsets:
        print("Itemset: ",itemset, " support:", support_finder(transactions, itemset))
    forced_breaker=1+forced_breaker
    if frequent_itemsets==[]:
        break
    else:
        previous_itemsets=create_next_level(frequent_itemsets)
        #here is a question, for next level I call create_next_level(frequent_itemsets), 
        #but frequent itemsets is generate with parameter previous_itemsets, so we use current level and should work
        
        #previous_itemsets = comparator(previous_itemsets, previous_itemsets_new_entries)
    if forced_breaker==10:
        break
    else:
        pass



Level 0 ------------------------------------------------
Itemset:  {'soup'}  support: 379
Itemset:  {'whole wheat pasta'}  support: 221
Itemset:  {'ham'}  support: 199
Itemset:  {'whole wheat rice'}  support: 439
Itemset:  {'cottage cheese'}  support: 239
Itemset:  {'pancakes'}  support: 713
Itemset:  {'frozen vegetables'}  support: 715
Itemset:  {'cookies'}  support: 603
Itemset:  {'french wine'}  support: 169
Itemset:  {'almonds'}  support: 153
Itemset:  {'fresh tuna'}  support: 167
Itemset:  {'honey'}  support: 356
Itemset:  {'olive oil'}  support: 494
Itemset:  {'tomato juice'}  support: 228
Itemset:  {'spaghetti'}  support: 1306
Itemset:  {'champagne'}  support: 351
Itemset:  {'energy drink'}  support: 200
Itemset:  {'ground beef'}  support: 737
Itemset:  {'herb & pepper'}  support: 371
Itemset:  {'hot dogs'}  support: 243
Itemset:  {'tomatoes'}  support: 513
Itemset:  {'cake'}  support: 608
Itemset:  {'cereals'}  support: 193
Itemset:  {'frozen smoothie'}  support: 475
Itemset:  

### Improved apriori

In [14]:
previous_itemsets=itemsets
forced_breaker=0
while True:
    print("Level",forced_breaker, "------------------------------------------------")
    frequent_itemsets, frequent_itemsets_values=find_frequent_itemsets(transactions, previous_itemsets, 150)
    confident_itemsets=find_confident_itemsets(transactions, frequent_itemsets, 0.3)
    if confident_itemsets==[]:
        pass
    else:
        frequent_itemsets=confident_itemsets
        
    for itemset in frequent_itemsets:
        print("Itemset: ",itemset, " support:", support_finder(transactions, itemset))
    forced_breaker=1+forced_breaker
    if frequent_itemsets==[]:
        break
    else:
        next_itemsets=create_next_level(frequent_itemsets)
        #difference, we apply the apriori_improvement, escentially leaving for every item of 2 levels, only the first alphabetical of 3 levels
        previous_itemsets=apriori_improvement(previous_itemsets, next_itemsets)
    if forced_breaker==10:
        break
    else:
        pass
print(previous_itemsets)

Level 0 ------------------------------------------------
Itemset:  {'soup'}  support: 379
Itemset:  {'whole wheat pasta'}  support: 221
Itemset:  {'ham'}  support: 199
Itemset:  {'whole wheat rice'}  support: 439
Itemset:  {'cottage cheese'}  support: 239
Itemset:  {'pancakes'}  support: 713
Itemset:  {'frozen vegetables'}  support: 715
Itemset:  {'cookies'}  support: 603
Itemset:  {'french wine'}  support: 169
Itemset:  {'almonds'}  support: 153
Itemset:  {'fresh tuna'}  support: 167
Itemset:  {'honey'}  support: 356
Itemset:  {'olive oil'}  support: 494
Itemset:  {'tomato juice'}  support: 228
Itemset:  {'spaghetti'}  support: 1306
Itemset:  {'champagne'}  support: 351
Itemset:  {'energy drink'}  support: 200
Itemset:  {'ground beef'}  support: 737
Itemset:  {'herb & pepper'}  support: 371
Itemset:  {'hot dogs'}  support: 243
Itemset:  {'tomatoes'}  support: 513
Itemset:  {'cake'}  support: 608
Itemset:  {'cereals'}  support: 193
Itemset:  {'frozen smoothie'}  support: 475
Itemset:  

## Eclat

In [15]:
def tid_finder(df, itemset):
    transactions=[]
    for i, row in enumerate(df):  
        if all(item in row for item in itemset): 
            transactions.append(i)  
    return transactions


def tid_lists_iterator(df, itemsets):
    t_id_list=[]
    for itemset in itemsets:
        t_ids=tid_finder(df, itemset)
        t_id_list.append((itemset, t_ids))
    return t_id_list

def intersect_finder(transactions_ids_list):
    intersections={}
    for i in range(len(transactions_ids_list)):
        #print("\n")
        itemset_i, transaction_i_id = transactions_ids_list[i]
        #print(itemset_i,"   ",transaction_i_id )
        for j in range(i + 1, len(transactions_ids_list)):
            itemset_j, transaction_j_id = transactions_ids_list[j]
            #print("   ",itemset_j,"   ",transaction_j_id )
            intersection = [transaction_id for transaction_id in transaction_i_id if transaction_id in transaction_j_id]
            #print(intersection)
            if intersection: 
                intersections[(tuple(itemset_i), tuple(itemset_j))] = intersection
    return intersections

list_with_id_tids_aux_testing=tid_lists_iterator(transactions, itemsets)
aux_testing=intersect_finder(list_with_id_tids_aux_testing)

In [16]:
#according to supplementary material:
#Step 1: Transform the dataset into TID lists
#Step 2: Compute intersections to find frequent itemsets

transactions_ids_list = tid_lists_iterator(transactions,itemsets)
frequents=[]
minimum_support= 100 #no specific indication
while True:
    intersections = intersect_finder(transactions_ids_list)
    current_level_itemsets = []
    
    for (itemset_i, itemset_j), intersection in intersections.items():
        support = len(intersection)
        if support>minimum_support:
            combined_itemset=sorted(set(itemset_i) | set(itemset_j))
            current_level_itemsets.append(combined_itemset)
            frequents.append((combined_itemset, support)) 
    if not current_level_itemsets:
        break
    transactions_ids_list = tid_lists_iterator(transactions, current_level_itemsets)
    print("Items and their intersection transactions")
    print(transactions_ids_list)


Items and their intersection transactions
[(['soup', 'spaghetti'], [25, 56, 140, 188, 202, 227, 305, 408, 453, 503, 540, 553, 665, 679, 808, 919, 988, 1248, 1330, 1400, 1414, 1420, 1539, 1571, 1604, 1636, 1848, 1866, 1879, 2064, 2197, 2287, 2395, 2489, 2541, 2551, 2552, 2625, 2643, 2654, 2700, 2744, 2791, 2814, 2871, 3076, 3082, 3167, 3281, 3283, 3445, 3470, 3494, 3495, 3520, 3527, 3558, 3589, 3805, 3846, 3855, 3908, 3919, 4578, 4709, 4752, 4838, 4889, 4924, 4936, 4949, 4969, 4971, 5096, 5128, 5172, 5175, 5189, 5217, 5261, 5281, 5421, 5521, 5570, 5597, 5602, 5645, 5684, 5853, 5885, 5887, 6036, 6078, 6113, 6149, 6179, 6312, 6520, 6566, 6622, 6659, 6677, 6765, 6915, 7077, 7330, 7476]), (['milk', 'soup'], [25, 56, 107, 118, 140, 146, 166, 305, 414, 437, 458, 503, 508, 514, 540, 565, 583, 620, 647, 666, 707, 794, 808, 842, 870, 872, 988, 1379, 1414, 1420, 1473, 1540, 1571, 1598, 1604, 1610, 1632, 1689, 1831, 1879, 1912, 2064, 2197, 2213, 2224, 2395, 2489, 2551, 2643, 2654, 2700, 2865, 3073